Data is pre-processed in Java by splitting it into sentences, each on a new line, and lowering the case of all files. New files are saved as TXT with "\_SENTbySENT" added at the end of the file name

In [36]:
# Import all necesssry modules
import os, sys, os.path
from stat import *
import errno
import glob
import csv
import re

## Count collected data files 

In [ ]:
#import os
import os.path

root = "/media/sf_Shared_Folder/Courses/Coursera Downloads Processed"
allFiles = sum([len(files) for r, d, files in os.walk("/media/sf_Shared_Folder/Courses/Coursera Downloads Processed")])

print("All files: ", allFiles)

# ---------------
def fileCounter(folder):    
    count = 0
    
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        
        if os.path.isfile(path):
            if os.path.isfile(path):
                name = os.path.basename(path)
                if name.endswith('.txt'):
                    count += 1
        elif os.path.isdir(path):
            count += fileCounter(path)
            
    return count
# ---------------

"""
TXT files:  563
SRT files:  568
RTF files:  562
All Files:  1693
"""

print("TXT files: ", fileCounter(root))

All files:  1693
TXT files:  563


## List files in directories

In [53]:
root = "/media/sf_Shared_Folder/Courses/Coursera Downloads Processed"
#files = glob.glob(root)

def traverseDir(rootIn):
# traverse root directory, and list directories as dirs and files as files
    for rootIn, dirs, files in os.walk("."):
        path = rootIn.split(os.sep)
        print((len(path) - 1) * '---', os.path.basename(root))
        for file in files:
            print(len(path) * '---', file)
            
traverseDir(root)

 Coursera Downloads Processed
--- TF-IDF.ipynb
--- ofile.txt
--- TF-IDF+NLTK+wordnet+wikipedia.ipynb
--- File processing.ipynb
--- Processing.ipynb
--- ifile.txt
--- ofileSent.csv
--- output.csv
--- ofileSent.txt
--- Coursera Downloads Processed
------ Coursera Downloads Processed
--------- Coursera Downloads Processed
------------ 01_what-are-the-benefits-of-sharing-data.en.txt
------------ 02_what-are-the-drawbacks-of-sharing-data.en.txt
--------- Coursera Downloads Processed
------------ 02_protecting-confidentiality-part-2.en.txt
------------ 04_access.en.txt
------------ 03_intellectual-property-and-data-ownership.en.txt
------------ 01_protecting-confidentiality-part-1.en.txt
--------- Coursera Downloads Processed
------------ 02_challenges-to-sharing.en.txt
------------ 03_data-citations.en.txt
------------ 01_benefits-of-sharing.en.txt
------ Coursera Downloads Processed
--------- Coursera Downloads Processed
------------ 01_welcome.en.txt
--------- Coursera Downloads Processed